In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions
from skimage.io import imread
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from transferlearning import *
import cv2
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
from PIL import Image


In [ ]:
import tensorflow as tf
model_path = "../model/face_model"
model = tf.keras.models.load_model(model_path)

In [2]:
def get_LIME_explanation(img, num_label, model, positive=False):
    
    img_name = img.split('/')[-1][:-4]
    savedir = "../lime_explanations/"
    savename_img = savedir+img_name+'_explained.jpg'
    savename_heatmap = savedir+img_name+'_heatmap.jpg'
    
    img_ = cv2.imread(img)
    img_ = cv2.cvtColor(img_, cv2.COLOR_BGR2RGB)
    img_ = img_ / 255
    img_ = np.expand_dims(img_, axis=0)
    
    explainer = lime_image.LimeImageExplainer()
    explanation = explainer.explain_instance(img_[0].astype('double'), model.predict, top_labels=int(num_label), hide_color=0, num_samples=100)

    temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=positive, num_features=int(num_label), hide_rest=False)
    explain_img = mark_boundaries(temp / 2 + 0.5, mask)
    ind =  explanation.top_labels[0]

    dict_heatmap = dict(explanation.local_exp[ind])
    heatmap = np.vectorize(dict_heatmap.get)(explanation.segments) 
    
    im1 = Image.fromarray((explain_img*255).astype(np.uint8))
    im1.save(savename_img)
    plt.imsave(savename_heatmap, heatmap)
    
    return savename_img, savename_heatmap

In [3]:
img = "C:/Users/KIIT/anaconda3/resize/image_resized.jpg"
label = 3
positive = False

model = tf.keras.models.load_model("../model/face_model")
img_path, heatmap_path = get_LIME_explanation(img, label, model,positive)

NameError: name 'tf' is not defined